## Load images

In [ ]:
from skimage.io import imread
from skimage.exposure import histogram
from skimage.color import rgb2gray
import glob as g
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis, entropy
%matplotlib inline

labels, labels_name = [], ['normal', 'ex']

normal = g.glob('./data_set/normal/*.png')
ex = g.glob('./data_set/ex/*.png')

images = []

for path in normal:
    im = imread(path, as_grey=True)
    images.append(im)
    labels.append(0)

for path in ex:
    im = imread(path, as_grey=True)
    images.append(im)
    labels.append(1)
    
plt.figure(figsize=(5, 10))
plt.imshow(images[70], cmap='gray')
plt.title('Image in grayscale')
plt.show()
plt.figure(figsize=(5, 10))
plt.imshow(images[69], cmap='gray')
plt.title('Image in grayscale')
plt.show()

## Extracting features

In [ ]:
dados = []

for i in images:
    features = []
    histo = histogram(i, nbins=256)[0]
    features.append(np.mean(histo))
    features.append(np.var(histo))
    features.append(skew(histo))
    features.append(kurtosis(histo))
    features.append(entropy(histo))
    features.append(np.power(histo, 2).sum())
    
    dados.append(features)

## Splitting data

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(dados, labels, test_size=0.2)

print('data train: {}'.format(len(train_data)))
print('data test: {}'.format(len(test_data)))
print('train labels: {}'.format(train_label))
print('test labels: {}'.format(test_label))
print('labels name: {}'.format(labels_name))

## Training

In [ ]:
from sklearn.svm import SVC

classifier = SVC()
classifier.fit(train_data, train_label)

## Test

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

prediction = classifier.predict(test_data)
accuracy = accuracy_score(test_label, prediction)
matrix = confusion_matrix(test_label, prediction)

plt.figure()
plot_confusion_matrix(matrix, classes=[labels_name[0], labels_name[1]], 
                      title='Confusion Matrix - Accuracy: {}'.format(accuracy))

plt.show()